<a href="https://colab.research.google.com/github/shivster2000/UROPs2025/blob/main/DialoGPT_Controllable_Complexity_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone repo and install dependencies

In [1]:
!git clone --single-branch --branch DialoGPT-Adaptation https://github.com/shivster2000/ControllableComplexityChatbot.git

Cloning into 'ControllableComplexityChatbot'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 55 (delta 21), reused 27 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 1.12 MiB | 5.08 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [13]:
!cd ControllableComplexityChatbot/

import sys
import os

# Specify the path to the directory you want to add
directory_to_add = '/content/ControllableComplexityChatbot/controllable_dialogpt/controllable_dialogpt.py' # Replace with your actual path

# Add the directory to sys.path
# It's generally good practice to add to the beginning so your modules
# are found before others with the same name in standard libraries.
if directory_to_add not in sys.path:
    sys.path.insert(0, directory_to_add)
    print(f"Added {directory_to_add} to sys.path")
else:
    print(f"{directory_to_add} is already in sys.path")

# You can print sys.path to verify the change
# print(sys.path)

/bin/bash: line 1: cd: ControllableComplexityChatbot/: No such file or directory
/content/ControllableComplexityChatbot/controllable_dialogpt/controllable_dialogpt.py is already in sys.path


In [16]:
cd ControllableComplexityChatbot/

[Errno 2] No such file or directory: 'ControllableComplexityChatbot/'
/content/ControllableComplexityChatbot


In [5]:
pip install numpy scipy regex torch transformers

In [10]:
!python demo.py

2025-07-24 11:09:02.780633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753355343.045218    8851 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753355343.117109    8851 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-24 11:09:03.698703: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/ControllableComplexityChatbot/demo.py", line 4, in <module>
    from cont

#Demo.py

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

from controllable_dialogpt import ControllableDialoGPT

from generation_utils import Reranker, Wordlist, cefr_to_int, load_wordlist

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
allowed_words = load_wordlist("data/sample_wordlist.txt")

# Replace agent_opt JSON with dict config
config = {
    "model_name": "microsoft/DialoGPT-medium",
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "top_k": 40,
    "top_p": 0.95,
    "num_return_sequences": 20,  # for reranking
    "control_method": "reranker",
    "reranker": Reranker(
        model="complexity_model",
        tokenizer="distilroberta-base",
        device="cuda" if torch.cuda.is_available() else "cpu",
        cefr=1
    )
    #"wordlist": Wordlist(allowed_words, tokenizer)

}

agent = ControllableDialoGPT(config)

print("Start chatting (type 'exit' to stop):")
while True:
    user_input = input(">> User: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    response = agent.generate_response(user_input)
    print(f"🤖 Bot: {response}")
agent.set_interactive_mode(True)


[DEBUG] config received: {'model_name': 'microsoft/DialoGPT-medium', 'device': 'cpu', 'top_k': 40, 'top_p': 0.95, 'num_return_sequences': 20, 'control_method': 'reranker', 'reranker': <generation_utils.Reranker object at 0x7f5fb143f410>}
[DEBUG] reranker set: <generation_utils.Reranker object at 0x7f5fb143f410>
[DEBUG] wordlist set: None
[DEBUG] init config keys: dict_keys(['model_name', 'device', 'top_k', 'top_p', 'num_return_sequences', 'control_method', 'reranker'])
[DEBUG] reranker in config: True
[DEBUG] wordlist in config: False
[DEBUG] control_method resolved: rerank
Start chatting (type 'exit' to stop):
>> User: exit


#controllable_dialogpt.py

In [25]:
!python controllable_dialogpt/controllable_dialogpt.py

2025-07-24 12:32:21.501265: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753360341.783604   30076 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753360341.864838   30076 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [1]:
import torch
import sys
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from generation_utils import Wordlist, Reranker, cefr_to_int, load_wordlist

class ControllableDialoGPT:
    def __init__(self, config):
        print("[DEBUG] config received:", config)
        self.config = config
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.history = None
        self.reranker = config.get("reranker", None)
        print("[DEBUG] reranker instance:", self.reranker)
        self.wordlist = config.get("wordlist", None)

        print("[DEBUG] reranker set:", self.reranker)
        print("[DEBUG] wordlist set:", self.wordlist)

        self.interactive_mode = config.get("interactive_mode", False)

        print("[DEBUG] init config keys:", config.keys())
        print("[DEBUG] reranker in config:", "reranker" in config)
        print("[DEBUG] wordlist in config:", "wordlist" in config)

        if self.reranker and self.wordlist:
            raise ValueError("Choose only one control method: reranker or wordlist.")

        self.control_method = "reranker" if self.reranker else "wordlist" if self.wordlist else "none"

        print("[DEBUG] control_method resolved:", self.control_method)

    def set_interactive_mode(self, mode: bool):
        self.interactive_mode = mode

    def generate_response(self, user_input):
        input_ids = self.tokenizer.encode(user_input + self.tokenizer.eos_token, return_tensors="pt").to(self.device)
        full_input = torch.cat([self.history, input_ids], dim=-1) if self.history is not None else input_ids

        attention_mask = torch.ones(full_input.shape, device=self.device)

        outputs = self.model.generate(
            full_input,
            attention_mask=attention_mask,
            max_length=1000,
            pad_token_id=self.tokenizer.eos_token_id,
            do_sample=True,
            top_k=40,
            top_p=0.95,
            temperature=0.7,
            num_return_sequences=20 if self.reranker else 1
        )


        candidates = [
            self.tokenizer.decode(out[full_input.shape[-1]:], skip_special_tokens=True)
            for out in outputs
        ]

        if self.control_method == "wordlist":
            candidates = [c for c in candidates if self._passes_vocab_filter(c)]
            print(f"candidates after wordlist filter: {candidates}")
            if not candidates:
                return "[No cheesy response]"

        if self.control_method == "reranker":
            print("[DEBUG] Using reranker for selection")
            best_response = self.reranker.rank(self, candidates)

        else:
            best_response = candidates[0]

        self.history = torch.cat([full_input, self.tokenizer.encode(best_response + self.tokenizer.eos_token, return_tensors="pt").to(self.device)], dim=-1)

        return best_response

    def _passes_vocab_filter(self, text: str) -> bool:

        tokens = text.strip().split()
        return all(word.lower().strip(".,!?") in self.wordlist.allowed_token_seqs for word in tokens)


ModuleNotFoundError: No module named 'generation_utils'

# generation_utils.py - move to main folder

In [24]:
import math
import os
import json
from typing import List, Set, Dict, Union, Optional

import numpy as np
import torch
from transformers import PreTrainedTokenizer
from transformers import RobertaForSequenceClassification, RobertaTokenizer

def cefr_to_int(cefr: str) -> int:
    mapping = {
        "A1": 0,
        "A2": 1,
        "B1": 2,
        "B2": 3,
        "C1": 4,
        "C2": 5,
    }
    clean_cefr = cefr.upper().strip()
    assert clean_cefr in mapping, f"CEFR must be one of {list(mapping.keys())}, not {cefr}"

    return mapping[clean_cefr]


def load_wordlist(path: str) -> List[str]:
    """
    Load a list of words from a text file containing one word per line
    """
    vocab = []

    if not path:
        return vocab

    assert os.path.isfile(path)

    with open(path, 'r', encoding="utf-8") as vocab_file:
        for row in vocab_file:
            token = row.strip()
            vocab.append(token)

    return vocab


class Wordlist():
    def __init__(self, allowed_words: List[str], tokenizer: PreTrainedTokenizer):
        self.tokenizer = tokenizer

        # Identify token ID sequences that are allowed words
        # Identify allowed continuations of sequences
        self.allowed_token_seqs: Set[str] = set()
        self.allowed_continuations: Dict[str, List[int]] = {}

        for word in allowed_words:
            for word_variant in self._get_word_variants(word):
                token_ids = tokenizer.encode(word_variant, add_special_tokens=False)
                if not token_ids:
                    continue

                self.allowed_token_seqs.add(repr(token_ids))

                for i in range(1, len(token_ids)):
                    prefix = repr(token_ids[:i])
                    next_token = token_ids[i]    # List represented as string for lookup
                    if prefix not in self.allowed_continuations:
                        self.allowed_continuations[prefix] = []
                    self.allowed_continuations[prefix].append(next_token)

    def get_allowed_ids(self, token_ids: List[int]) -> List[int]:
        """
        adapts parlai-based _get_continuation_ids function
        """
        last_word = self._get_last_word(token_ids)
        prefix_str = repr(last_word)
        continuation_ids = self.allowed_continuations.get(prefix_str, [])

        if self._is_word(last_word) or not last_word:
          continuation_ids += self._get_starting_tokens()

        return list(set(continuation_ids))


    def _is_word(self, token_ids: List[int]) -> bool:
        """
        For a given sequence of token IDs, determine whether that sequence is a complete word
        """
        return repr(token_ids) in self.allowed_token_seqs


    def _get_last_word(self, token_ids: List[int]) -> List[int]:
        """
        Get the sequence of token IDs after the last word boundary.
        Assumes that a word boundary is denoted by punctuation or whitespace (Ġ).
        """

        if not token_ids:
          return []

        tokens = self.tokenizer.convert_ids_to_tokens(token_ids)
        for i in range(len(token_ids) -1, -1, -1):
            if tokens[i].startswith("Ġ") or tokens[i].startswith("_"):
                return token_ids[i:]
        return token_ids


    def _get_starting_tokens(self) -> List[int]:
        """
        return token IDs that can start a word (e.g. Ġ or_ initial)
        """
        vocab = self.tokenizer.get_vocab()
        starting_tokens = [idx for tok, idx in vocab.items() if tok.startswith("Ġ") or tok.startswith("_") or tok.isalpha()]
        return starting_tokens


    def _get_word_variants(self, word: str) -> Set[str]:
        return {word, word.lower(), word.capitalize()}



class Reranker():
    def __init__(self,
                 cefr: int,
                 model: str,
                 tokenizer: str = "distilroberta-base",
                 device: Optional[str] = "cuda",
                 text_truncate: int = 128,
                 exempt_tokens: Union[str, List[int]] = "all",
                 penalty_stddev: int = 2,
                 vocab_size: int = 8008,
                 word_filter: Optional[List[str]] = None):

        self.tokenizer = RobertaTokenizer.from_pretrained(tokenizer)
        self.model = RobertaForSequenceClassification.from_pretrained(model)
        self.model.to(device)
        self.device = device

        self.target_cefr = cefr
        self.text_truncate = text_truncate
        self.word_filter = word_filter

        cefr_filepath = os.path.join(os.path.dirname(__file__), 'tokens_by_cefr.json')
        with open(cefr_filepath, 'r') as cefr_file:
            token_cefrs = json.load(cefr_file)

        if exempt_tokens == "all" or penalty_stddev < 0:      # No penalties
            self.token_penalties = torch.tensor([[1] * vocab_size])
        else:
            # calculate penalties per CEFR level difference (0 = same CEFR)
            normal_dist = torch.distributions.normal.Normal(0, penalty_stddev)
            cefr_penalties = [math.exp(normal_dist.log_prob(torch.tensor(i))) for i in range(6)]

            token_penalties = []
            for i in range(vocab_size):
                if i in exempt_tokens:
                    token_penalties.append(cefr_penalties[0])

                elif str(i) in token_cefrs:
                    token_str, token_cefr = token_cefrs[str(i)]
                    penalty = cefr_penalties[int(token_cefr - self.target_cefr)]

                    if token_cefr <= self.target_cefr or not token_str.isalpha():         # ignore lower CEFR levels and punctuation/special tokens
                        penalty = cefr_penalties[0]

                    token_penalties.append(penalty)

                else:       # Assume highest CEFR level if we don't have an assigned CEFR level
                    token_penalties.append(cefr_penalties[int(5 - self.target_cefr)])

            self.token_penalties = torch.tensor([token_penalties])

    def get_complexity_scores(self, hyps: List[str]) -> np.ndarray:
        model_inputs = self.tokenizer(hyps,
                                      padding='max_length',
                                      truncation=True,
                                      max_length=self.text_truncate,
                                      return_tensors='pt',
                                      return_token_type_ids=True,
                                      return_attention_mask=True)

        model_output = self.model(input_ids=model_inputs["input_ids"].to(self.device),
                                  attention_mask=model_inputs["attention_mask"].to(self.device),
                                  token_type_ids=model_inputs["token_type_ids"].to(self.device))

        complexity_scores = model_output.logits.cpu().numpy().flatten()
        complexity_diffs = 5 - np.absolute(complexity_scores - self.target_cefr)      # reversed so that higher score = better

        return complexity_diffs

    def rank(self, candidates: List[str]) -> str:
        print("[DEBUG] rank() called with", len(candidates), "candidates")
        scores = self.get_complexity_scores(candidates)
        print("\n[DEBUG] Reranker scores:")
        for i, (text, score) in enumerate(zip(candidates, scores)):
            print(f"Candidate {i}: Score={score:.2f} -> {text}")

        best_index = scores.argmax()
        return candidates[best_index]



# _init.py

In [ ]:
from .controllable_dialogpt import ControllableDialoGPT
